In [1]:
import torch

import zipfile
from pathlib import Path

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm
import datetime
import random
import cv2
import src.models as mdls
import src.dataloaders as dataloaders
import src.utils as utils

In [2]:
config = utils.get_options()

## for google colab

In [3]:
model = mdls.get_pretrained_VGG16()
dataloader = dataloaders.dataloader

/cephfs/projects/ppashin/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/cephfs/projects/ppashin/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# print(len(dataloaders.dataloader))
# print(len(dataloaders.dataset_for_detection))
# print(dataloaders.dataset_of_backgrounds[89])

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
# Define a directory to save the checkpoints
checkpoint_dir = "checkpoints/"

# Ensure the directory exists
import os
os.makedirs(checkpoint_dir, exist_ok=True)

In [7]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.to(device)
loss_fn.to(device)
None

MSELoss()

In [8]:
print(len(dataloader))
for batch in dataloader:
    # print(batch[0])
    print(batch[1])
    break

625
tensor([[ 0, -1, -1, -1, -1],
        [ 0, -1, -1, -1, -1],
        [ 0, -1, -1, -1, -1],
        [ 0, -1, -1, -1, -1]])


In [9]:
epochs = config['epochs']

for epoch in range(epochs):
    epoch_loss = 0.0
    for sample in (pbar := tqdm(dataloader)):

        img, box = img, box = sample[0].to(device), sample[1].to(device)
        img = img.to(torch.float32)

        box = box.to(torch.float32)
        optimizer.zero_grad()
        pred = model(img)

        loss = loss_fn(pred, box)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch: {epoch}\tLoss: {epoch_loss / len(dataloader)}")
    checkpoint_filename = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth")
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss,
    }, checkpoint_filename)

100%|██████████| 625/625 [01:33<00:00,  6.65it/s]


Epoch: 0	Loss: 0.6588981464379468


100%|██████████| 625/625 [01:29<00:00,  6.99it/s]


Epoch: 1	Loss: 2.220576684730407


100%|██████████| 625/625 [01:28<00:00,  7.03it/s]


...

Epoch: 7	Loss: 0.007008948312373832


100%|██████████| 625/625 [01:29<00:00,  7.01it/s]


Epoch: 8	Loss: 0.00546949240071699


100%|██████████| 625/625 [01:28<00:00,  7.03it/s]


Epoch: 9	Loss: 0.004488049800088629


In [ ]:
max_split_size_mb